In [152]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
#Display Max cols and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Disable warning
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading the Data into a Data Frame

In [153]:
df=pd.read_csv('googleplaystore.csv')


## Exploring the data

In [154]:


df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [155]:
print(f'The Number of rows are {df.shape[0]}, and {df.shape[1]} columns')
print(f'Names of the colums:\n {df.columns}')
df.info()

The Number of rows are 10841, and 13 columns
Names of the colums:
 Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  int64  
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10841 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 

In [156]:
df.describe()


,Rating,Reviews
count,9367.000000,1.084100e+04
mean,4.191513,4.441119e+05
std,0.515735,2.927629e+06
min,1.000000,0.000000e+00
25%,4.000000,3.800000e+01
50%,4.300000,2.094000e+03
75%,4.500000,5.476800e+04
max,5.000000,7.815831e+07


__________________________
#### Size should also be a numeric as Ratings and Reviews, but its data type is object, so I guess should convert it to the float.
__________________________________

#### So let's examine the rows of Column Size

In [157]:
df['Size']

0                       19M
1                       14M
2                      8.7M
3                       25M
4                      2.8M
5                      5.6M
6                       19M
7                       29M
8                       33M
9                      3.1M
10                      28M
11                      12M
12                      20M
13                      21M
14                      37M
15                     2.7M
16                     5.5M
17                      17M
18                      39M
19                      31M
20                      14M
21                      12M
22                     4.2M
23                     7.0M
24                      23M
25                     6.0M
26                      25M
27                     6.1M
28                     4.6M
29                     4.2M
30                     9.2M
31                     5.2M
32                      11M
33                      11M
34                     4.2M
35                  

#### There are three different string/characters that are stopping it to be numeric
1. 'M' which represents the filesize in Mb
2. 'k' which represents the filesize in Kb
3. 'Varies with device' That represents variable file sizes for different versions of Android.

_____________________
#### The least unit in the Size column is kb, let's convert that into Mb so we can have not very high values
- A function examine and convert the supplied value can perform the task when applied on all the rows one by one. So, writing that.


In [158]:
def conv2kb(size):
    if isinstance(size,str):
        if 'k' in size:
            return format(float(str(size).replace('k',""))/1024,'.3f')
        if 'M' in size:
            return float(str(size).replace('M',""))
    else: 
        return(size)



#### Now let's apply the function on the data and see what ir returns

- We will impute the 'varies with device' by the mean of the column

In [159]:
a=0
df['Size']=df['Size'].str.replace('Varies with device','0.00')  ## replacing the string with a placeholder float, so we can calculate the mean value and since float of 0.00 will become NaN so we can replace it using fillna.
df['Size']=df['Size'].apply(lambda x: conv2kb(x)) #  df['Size'].loc[a]=conv2kb(df['Size'].loc[a])
# now as the column is of object data type and we can't calculate the mean of it, so changing the data type to float so the mean can be calculated and imputed in rows with 0.00 value
df['Size']=pd.to_numeric(df['Size'], errors='raise')
df['Size'].fillna(format(df['Size'].mean(), '.1f'), inplace=True )
df['Size']=pd.to_numeric(df['Size'], errors='raise')
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  int64  
 4   Size            10841 non-null  float64
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10841 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10839 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.1+ MB


#### Since the column has been updated as per our plan and converted into float, now let's rename it to size_Mb so we can remember that all the values in the column are in Megabytes.

In [160]:
# Now in order to keep it in the record, that all Size rows are defined in Kb we will rename to columns.
df.rename(columns={"Size":"size_Mb"}, errors="raise", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  int64  
 4   size_Mb         10841 non-null  float64
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10841 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10839 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.1+ MB


#### Objective for Size is achieved here, let's move further

_____________________

### Dealing with Installs Columns

In [161]:
df['Installs'].unique()


array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+', '0'], dtype=object)

   #### Operations to be performed on Installs
1. Remove the '+' Sign
2. converting it to the integer

In [162]:
df['Installs']=df['Installs'].apply(lambda x: x.replace('+', ""))
df['Installs']=df['Installs'].apply(lambda x: x.replace(',', ""))

In [163]:
df['Installs'].unique()

array(['10000', '500000', '5000000', '50000000', '100000', '50000',
       '1000000', '10000000', '5000', '100000000', '1000000000', '1000',
       '500000000', '50', '100', '500', '10', '1', '5', '0'], dtype=object)

In [164]:
df['Installs']=df['Installs'].apply(lambda x: int(x))

#### Now let's make a bin for Installs. The same needs to be done with reviews.

In [165]:
df['Install_bins'] = pd.cut(x=df['Installs'], bins=[-1, 1000, 100000, 10000000,100000000,1000000000,], labels=[1,2,3,4,5])
df['Install_bins']=pd.to_numeric(df['Install_bins'], errors='raise')
# Reviews Bins Might also be required
df['Reviews_bins'] = pd.cut(x=df['Reviews'], bins=[-1, 100, 10000, 100000,1000000,100000000,], labels=[1,2,3,4,5])
df['Reviews_bins']=pd.to_numeric(df['Install_bins'], errors='raise')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  int64  
 4   size_Mb         10841 non-null  float64
 5   Installs        10841 non-null  int64  
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10841 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10839 non-null  object 
 13  Install_bins    10841 non-null  int64  
 14  Reviews_bins    10841 non-null  int64  
dtypes: float64(2), int64(4), object(9)
memory usage: 1.2+ MB


_________________________
### Now let's convert price into Numeric

In [166]:
df['Price'].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$1.49', '$2.99', '$7.99', '$5.99',
       '$3.49', '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49',
       '$10.00', '$24.99', '$11.99', '$79.99', '$16.99', '$14.99',
       '$1.00', '$29.99', '$12.99', '$2.49', '$10.99', '$1.50', '$19.99',
       '$15.99', '$33.99', '$74.99', '$39.99', '$3.95', '$4.49', '$1.70',
       '$8.99', '$2.00', '$3.88', '$25.99', '$399.99', '$17.99',
       '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$2.50',
       '$1.59', '$6.49', '$1.29', '$5.00', '$13.99', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$19.90', '$8.49', '$1.75',
       '$14.00', '$4.85', '$46.99', '$109.99', '$154.99', '$3.08',
       '$2.59', '$4.80', '$1.96', '$19.40', '$3.90', '$4.59', '$15.46',
       '$3.04', '$4.29', '$2.60', '$3.28', '$4.60', '$28.99', '$2.95',
       '$2.90', '$1.97', '$200.00', '$89.99', '$2.56', '$30.99', '$3.61',
       '$394.99', '$1.26', '$1.20', '$1.04'], dtype=object)

#### Let's remove the $ size and convert it to the float

In [167]:
df['Price']=df['Price'].apply(lambda x: x.replace('$', ''))
df['Price']=pd.to_numeric(df['Price'], errors='raise')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  int64  
 4   size_Mb         10841 non-null  float64
 5   Installs        10841 non-null  int64  
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  float64
 8   Content Rating  10841 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10839 non-null  object 
 13  Install_bins    10841 non-null  int64  
 14  Reviews_bins    10841 non-null  int64  
dtypes: float64(3), int64(4), object(8)
memory usage: 1.2+ MB


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  int64  
 4   size_Mb         10841 non-null  float64
 5   Installs        10841 non-null  int64  
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  float64
 8   Content Rating  10841 non-null  object 
 9   Genres          10840 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10839 non-null  object 
 13  Install_bins    10841 non-null  int64  
 14  Reviews_bins    10841 non-null  int64  
dtypes: float64(3), int64(4), object(8)
memory usage: 1.2+ MB


_____
## Now Let's Check and Fill the Nulls

In [169]:
df.describe()

,Rating,Reviews,size_Mb,Installs,Price,Install_bins,Reviews_bins
count,9367.000000,1.084100e+04,10841.00000,1.084100e+04,10841.000000,10841.000000,10841.000000
mean,4.191513,4.441119e+05,21.51193,1.546291e+07,1.027273,2.294807,2.294807
std,0.515735,2.927629e+06,20.74761,8.502557e+07,15.948971,0.953426,0.953426
min,1.000000,0.000000e+00,0.00800,0.000000e+00,0.000000,1.000000,1.000000
25%,4.000000,3.800000e+01,5.90000,1.000000e+03,0.000000,1.000000,1.000000
50%,4.300000,2.094000e+03,18.00000,1.000000e+05,0.000000,2.000000,2.000000
75%,4.500000,5.476800e+04,26.00000,5.000000e+06,0.000000,3.000000,3.000000
max,5.000000,7.815831e+07,100.00000,1.000000e+09,400.000000,5.000000,5.000000


In [170]:
df.isnull().sum()


App                  0
Category             0
Rating            1474
Reviews              0
size_Mb              0
Installs             0
Type                 1
Price                0
Content Rating       0
Genres               1
Last Updated         0
Current Ver          8
Android Ver          2
Install_bins         0
Reviews_bins         0
dtype: int64

### Deleting rows from the df which are nan in Current Ver, Android Ver Genres and Type, as these are less than 0.1%


In [171]:
df=df[~df['Current Ver'].isna() & ~df['Genres'].isna() & ~df['Type'].isna() & ~df['Android Ver'].isna()]
print('Nulls summary: \n',df.isnull().sum())
df.head(10)


Nulls summary: 
 App                  0
Category             0
Rating            1469
Reviews              0
size_Mb              0
Installs             0
Type                 0
Price                0
Content Rating       0
Genres               0
Last Updated         0
Current Ver          0
Android Ver          0
Install_bins         0
Reviews_bins         0
dtype: int64


,App,Category,Rating,Reviews,size_Mb,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Install_bins,Reviews_bins
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,2,2
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,3,3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,3,3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,4,4
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2,2
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6,50000,Free,0.0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,2,2
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19.0,50000,Free,0.0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,2,2
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29.0,1000000,Free,0.0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,3,3
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33.0,1000000,Free,0.0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up,3,3
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1,10000,Free,0.0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up,2,2


__________
### Initial Assessment:
- Ratings have plenty of Nans around 20% that needs to be filled. 
- Nans do not belong to single category or price or size or installs group but distribution is pretty normallized accross the data. So we can't fill it with simple Mean, Median or Mode etc.
### Tackling Plan:
- We will see the correlation between the different columns.
- Identify the relationship between different columns and decide.
- If required will use ML/DL to fill the missing data.

In [172]:
df['Reviews'].max()

78158306

In [173]:
#Let's separate the relevant data to do the further analysis
#df2=pd.DataFrame([df.Installs,df.Reviews, df.Price, df.Rating]).transpose()
df2=df[['Installs', 'Reviews', 'Price', 'Rating','Install_bins','Reviews_bins']]
# Creating another dataframe without nulls for analysis
df3=df2.dropna()




In [174]:
print('Installation Bins: \n',df2['Install_bins'].value_counts())
print('Group by: \n',df2.groupby(df2['Install_bins'])['Rating'].describe())
print("Correlation: \n",df2.corr())
print("Mean \n", df2.groupby(df2['Install_bins'])['Rating'].mean())

Installation Bins: 
 Install_bins
3    4120
2    3175
1    2706
4     698
5     130
Name: count, dtype: int64
Group by: 
                count      mean       std  min  25%  50%  75%  max
Install_bins                                                     
1             1359.0  4.198749  0.833807  1.0  3.9  4.4  4.8  5.0
2             3056.0  4.066001  0.555036  1.4  3.8  4.2  4.5  5.0
3             4117.0  4.246247  0.341577  1.8  4.1  4.3  4.5  4.9
4              698.0  4.386533  0.192817  3.1  4.3  4.4  4.5  4.8
5              130.0  4.309231  0.186126  3.7  4.2  4.3  4.4  4.7
Correlation: 
               Installs   Reviews     Price    Rating  Install_bins  \
Installs      1.000000  0.643112 -0.011698  0.051337      0.411386   
Reviews       0.643112  1.000000 -0.009674  0.068133      0.316223   
Price        -0.011698 -0.009674  1.000000 -0.021919     -0.053502   
Rating        0.051337  0.068133 -0.021919  1.000000      0.125543   
Install_bins  0.411386  0.316223 -0.053502  0.12554

## Not Much Correlation between the colums.

- Max correlation between Ratings and any other column is less than 13%
- Machine Learning algorithms (regression) might not generate accurate results.
- Similarly Deep Learning algorithms might not have enough features to predict accurately.
- The best way is that as Installs and Reviews are correlated so we should statistically fill the Nans.
- But when we inspect the mean grouped by the Installs_bin, not much deviation between each bin. So I guess total mean will fall in the same range 4.19-4.38
- So filling the nan's with mean should do the task

In [175]:
print('Mean: ', df['Rating'].mean())
df['Rating'].fillna(df['Rating'].mean(), inplace=True)
print(df['Rating'].isnull().sum())

Mean:  4.191837606837606
0


_____
### Now let's visualize the data

#### Price Vs Ratings

In [176]:
px.scatter(x=df['Category'],y=df['Price'], color=df['Rating'])

In [177]:
px.bar(x=df['Rating'], y=df['Price'],color=df['Category'])

In [178]:
px.bar(x=df['Category'].unique(), y=df['Price'].groupby(df['Category']).mean(), color=df['Installs'].groupby(df['Category']).sum())

In [179]:
px.sunburst(df, path=['Category', 'Rating', 'Reviews'], values='Installs', color='Android Ver')

In [180]:

px.pie(x=df['Category'].unique(), y=df['Installs'].groupby(df['Category']).mean(), color=df['Rating'].groupby(df['Category']).mean())

TypeError: pie() got an unexpected keyword argument 'x'